In [1]:
!pip install torch transformers peft datasets bitsandbytes
!pip install --upgrade bitsandbytes

In [4]:
# from datasets import load_dataset
# from transformers import (

#     BitsAndBytesConfig

# )
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from peft import get_peft_model, LoraConfig, TaskType
# from datasets import load_dataset
# from peft import prepare_model_for_kbit_training
# import time
# import bitsandbytes as bnb
# import os
# import warnings
# warnings.filterwarnings('ignore')

# # Step 1: Load the dataset
# def load_snli_dataset():
#     dataset = load_dataset("snli")
#     train_data = dataset['train'].select(list(range(0, 550000, 550)))  # 1000 samples
#     val_data = dataset['validation'].select(list(range(0, 10000, 100)))  # 100 samples
#     test_data = dataset['test'].select(list(range(0, 10000, 100)))  # 100 samples
#     return train_data, val_data, test_data

# # Step 2: Load the model and tokenizer with bitsandbytes for 4-bit
# def load_model_and_tokenizer():
#     model_name = "microsoft/phi-2"
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     # Add a padding token if it's not already set
#     if tokenizer.pad_token is None:
#         tokenizer.pad_token = tokenizer.eos_token  # Set pad token to be the same as eos token

#     # Load model with bitsandbytes in 4-bit
#     model = AutoModelForSequenceClassification.from_pretrained(
#         model_name,
#         num_labels=3,  # 3 labels for NLI
#         load_in_4bit=True,  # Enable 4-bit loading
#         device_map='auto'  # Automatically place model on GPU
#     )

#     # Set the pad token ID in the model's configuration
#     model.config.pad_token_id = tokenizer.pad_token_id

#     return model, tokenizer

# # Step 3: Prepare model for QLoRA
# # Step 3: Prepare model for QLoRA
# def prepare_model(model):
#     # Configure LoRA
#     lora_config = LoraConfig(
#         r=16,
#         lora_alpha=32,
#         lora_dropout=0.1,
#         task_type=TaskType.SEQ_CLS,
#     )
    
#     # Prepare the model for k-bit training
#     model = prepare_model_for_kbit_training(model)  # Removed use_4bit
#     model = get_peft_model(model, lora_config)
#     return model

# # Step 4: Training loop
# from torch.utils.data import DataLoader

# # Step 4: Training loop with DataLoader
# def train_model(model, train_data, val_data, tokenizer, epochs=5, batch_size=8):
#     model.train()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

#     # Create DataLoader for training data
#     train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


#     for epoch in range(epochs):
#         epoch_start_time = time.time()
#         for batch in train_loader:
#             inputs = tokenizer(batch['premise'], batch['hypothesis'], padding=True, truncation=True, return_tensors="pt")
#             labels = torch.tensor(batch['label']).to(device)
#             inputs = {k: v.to(device) for k, v in inputs.items()}
#             outputs = model(**inputs, labels=labels)
#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()
        
#         # Save the model after every epoch
#         model_save_path = f"fine_tuned_model_epoch_{epoch + 1}"
#         model.save_pretrained(model_save_path)
#         print(f"Epoch {epoch + 1} completed. Model saved to {model_save_path}.")
#         print(f"Time taken for epoch {epoch + 1}: {time.time() - epoch_start_time:.2f} seconds")

#         # Evaluate on validation set (optional)
#         evaluate_model(model, val_data, tokenizer)  # Add this line if you want to validate after each epoch

# # Step 5: Evaluation
# def evaluate_model(model, test_data, tokenizer):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     correct_predictions = 0
    
#     with torch.no_grad():
#         for batch in test_data:
#             inputs = tokenizer(batch['premise'], batch['hypothesis'], padding=True, truncation=True, return_tensors="pt")
#             inputs = {k: v.to(device) for k, v in inputs.items()}
#             outputs = model(**inputs)
#             predictions = outputs.logits.argmax(dim=-1)
#             if predictions.item() == batch['label']:
#                 correct_predictions += 1

#     accuracy = correct_predictions / len(test_data)
#     print(f"Accuracy on test set: {accuracy:.4f}")

# # Step 6: Main function
# def main():
#     train_data, val_data, test_data = load_snli_dataset()
#     model, tokenizer = load_model_and_tokenizer()
#     model = prepare_model(model)
#     # print("Accuracy before finetuning: \n")
#     # evaluate_model(model, test_data, tokenizer)  # Add this line if you want to validate after each epoch

#     train_model(model, train_data, val_data, tokenizer)
#     # print("Final accuracy on test set:\n")
#     evaluate_model(model, test_data, tokenizer)

# if __name__ == "__main__":
#     main()
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
from peft import prepare_model_for_kbit_training
import torch
from torch.utils.data import DataLoader
import time
import bitsandbytes as bnb
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Step 1: Load the dataset
def load_snli_dataset():
    dataset = load_dataset("snli")
    train_data = dataset['train'].select(list(range(0, 550000, 550)))  # 1000 samples
    val_data = dataset['validation'].select(list(range(0, 10000, 100)))  # 100 samples
    test_data = dataset['test'].select(list(range(0, 10000, 100)))  # 100 samples
    return train_data, val_data, test_data

# Step 2: Load the model and tokenizer with bitsandbytes for 4-bit
def load_model_and_tokenizer():
    model_name = "microsoft/phi-2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Add a padding token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Set pad token to be the same as eos token

    # Load model with bitsandbytes in 4-bit
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3,  # 3 labels for NLI
        load_in_4bit=True,  # Enable 4-bit loading
        device_map='auto'  # Automatically place model on GPU
    )

    # Set the pad token ID in the model's configuration
    model.config.pad_token_id = tokenizer.pad_token_id

    return model, tokenizer

# Step 3: Prepare model for QLoRA
def prepare_model(model):
    # Configure LoRA
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        task_type=TaskType.SEQ_CLS,
    )
    
    # Prepare the model for k-bit training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    return model

# Step 4: Training loop
def train_model(model, train_data, val_data, tokenizer, epochs=5, batch_size=8):
    model.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Create DataLoader for training data
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        epoch_start_time = time.time()
        for batch in train_loader:
            inputs = tokenizer(batch['premise'], batch['hypothesis'], padding=True, truncation=True, return_tensors="pt")
            # Move labels directly to device
            labels = torch.tensor(batch['label']).to(device)  # Ensure labels are a tensor
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Forward pass
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Save the model after every epoch
        model_save_path = f"fine_tuned_model_epoch_{epoch + 1}"
        model.save_pretrained(model_save_path)
        print(f"Epoch {epoch + 1} completed. Model saved to {model_save_path}.")
        print(f"Time taken for epoch {epoch + 1}: {time.time() - epoch_start_time:.2f} seconds")

        # Evaluate on validation set (optional)
        evaluate_model(model, val_data, tokenizer)  # Add this line if you want to validate after each epoch

# Step 5: Evaluation
def evaluate_before_finetune(model, test_data, tokenizer):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in test_data:
            inputs = tokenizer(batch['premise'], batch['hypothesis'], padding=True, truncation=True, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            predictions = outputs.logits.argmax(dim=-1)
            if predictions.item() == batch['label']:
                correct_predictions += 1

    accuracy = correct_predictions / len(test_data)
    print(f"Accuracy on test set before fine-tuning: {accuracy:.4f}")

def evaluate_after_finetune(model, test_data, tokenizer):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in test_data:
            inputs = tokenizer(batch['premise'], batch['hypothesis'], padding=True, truncation=True, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            predictions = outputs.logits.argmax(dim=-1)
            if predictions.item() == batch['label']:
                correct_predictions += 1

    accuracy = correct_predictions / len(test_data)
    print(f"Final accuracy on test set after fine-tuning: {accuracy:.4f}")

    
def evaluate_model(model, test_data, tokenizer):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in test_data:
            inputs = tokenizer(batch['premise'], batch['hypothesis'], padding=True, truncation=True, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            predictions = outputs.logits.argmax(dim=-1)
            if predictions.item() == batch['label']:
                correct_predictions += 1

    accuracy = correct_predictions / len(test_data)
    print(f"Accuracy on test set: {accuracy:.4f}")

# Step 6: Main function
def main():
    train_data, val_data, test_data = load_snli_dataset()
    model, tokenizer = load_model_and_tokenizer()
    model = prepare_model(model)

    evaluate_before_finetune(model, test_data, tokenizer)

    train_model(model, train_data, val_data, tokenizer)

    evaluate_after_finetune(model, test_data, tokenizer)

if __name__ == "__main__":
    main()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy on test set before fine-tuning: 0.2900
Epoch 1 completed. Model saved to fine_tuned_model_epoch_1.
Time taken for epoch 1: 203.71 seconds
Accuracy on test set: 0.6000
Epoch 2 completed. Model saved to fine_tuned_model_epoch_2.
Time taken for epoch 2: 135.31 seconds
Accuracy on test set: 0.7600
Epoch 3 completed. Model saved to fine_tuned_model_epoch_3.
Time taken for epoch 3: 135.73 seconds
Accuracy on test set: 0.7600
Epoch 4 completed. Model saved to fine_tuned_model_epoch_4.
Time taken for epoch 4: 134.58 seconds
Accuracy on test set: 0.8000
Epoch 5 completed. Model saved to fine_tuned_model_epoch_5.
Time taken for epoch 5: 134.98 seconds
Accuracy on test set: 0.7500
Final accuracy on test set after fine-tuning: 0.7500


In [5]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def count_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return trainable_params

# Call these functions after loading your model
model, tokenizer = load_model_and_tokenizer()
model = prepare_model(model)

# Counting parameters
total_params = count_parameters(model)
trainable_params = count_trainable_parameters(model)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total parameters: 1408634880
Trainable parameters: 18357760
